# Chapter 16. Operator Overloading

Trong Python, có thể tính lãi kép bằng cách:
- `interest = principal * ((1 + rate) ** periods - 1)`
  - Các toán tử xuất hiện giữa các toán hạng (+,*,-) được gọi là `infix operators`, nó có thể xử lý các loại tuỳ ý
    - Nhưng trong Java, nếu  chuyển từ float sang BigDecimal để có kết quả chính xác, thì bạn không thể sử dụng `infix operators` nữa, vì chúng chỉ hoạt động với kiểu nguyên thủy

Also, in Example 11-2, we noted that the `Vector2d.__eq__` method considers this to be `True`: `Vector(3, 4) == [3, 4]`—which may or not make sense. We will address these matters in this chapter, as well as:

- Cách một infix operator báo hiệu nó không thể xử lý một toán hạng

- Sừ dụng `duck typing` or `goose typing` để xử lý nhiều loại toán hạng

- The special behavior of the rich comparison operators (e.g., ==, >, <=, etc.)

- The default handling of augmented assignment operators such as +=, and how to overload them

## Operator Overloading 101

Đây là là một tính năng ngôn ngữ có thể (và đã) bị lạm dụng, dẫn đến sự nhầm lẫn của lập trình viên, lỗi và tắc nghẽn hiệu suất không mong muốn. Nhưng nếu dùng tốt thì code/API sẽ dễ đọc
- Cannot create new operator, only overload existing one
- A few operators can’t be overloaded: `is`, `and`, `or`, `not` (but the bitwise `&,` `|`, `~`, can).

## Unary (1 ngôi) Operators

`-`, implemented by `__neg__`
Phủ định. If `x == -2` then `-x == 2`.

`+`, implemented by `__pos__`
Khẳng định. Usually x == +x, but there are a few cases when that’s not true (https://learning.oreilly.com/library/view/fluent-python-2nd/9781492056348/ch16.html#idm46582410760976)

`~`, implemented by `__invert__`
Nghịch đảo 1 số nguyên, defined as ~x == -(x+1). If x == 2(10) then ~x == -3

abs, implemented by `__abs__`

In [1]:
from array import array
import reprlib
import math
import functools
import operator
import itertools


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    # tag::VECTOR_V6_UNARY[]
    def __abs__(self):
        return math.hypot(*self)

    def __neg__(self):
        return Vector(-x for x in self)  # <1>

    def __pos__(self):
        return Vector(self)  # <2>
    # end::VECTOR_V6_UNARY[]

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]

    __match_args__ = ('x', 'y', 'z', 't')

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)

    def angle(self, n):
        r = math.hypot(*self[n:])
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    # tag::VECTOR_V6_ADD[]
    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented

    def __radd__(self, other): # Kiểu như nếu vector2d + vector,  ko có vector2d.add(vector) --> nó sẽ gọi vector.radd(vector2d)
        return self + other
# end::VECTOR_V6_ADD[]

In [2]:
v1 = Vector([3, 4])

-v1, +v1, abs(v1)

(Vector([-3.0, -4.0]), Vector([3.0, 4.0]), 5.0)

## Overloading + for Vector Addition

The Vector class is a sequence type and sequences should support the `+` operator for concatenation and `*` for repetition

TIP
If users want to concatenate or repeat Vector instances, they can convert them to tuples or lists, apply the operator, and convert back—thanks to the fact that Vector is iterable and can be constructed from an iterable:
```
>>> v_concatenated = Vector(list(v1) + list(v2))
>>> v_repeated = Vector(tuple(v1) * 5)
```

In [3]:
v1 = Vector([3, 4, 5])
v2 = Vector([6, 7, 8])
v1 + v2

Vector([9.0, 11.0, 13.0])

In [4]:
v1 = Vector([3, 4, 5, 6])
v3 = Vector([1, 2])
v1 + v3

Vector([4.0, 6.0, 5.0, 6.0])

Therefore, to make the mixed-type additions, we need to implement the `Vector.__radd__` method, which Python will invoke as a fallback if the left operand does not implement `__add__`, or if it does but returns NotImplemented to signal that it doesn’t know how to handle the right operand.

![](image/16-1.png)

In [5]:
(10, 20, 30) + v1

Vector([13.0, 24.0, 35.0, 6.0])

In [6]:
v1 +(10, 20, 30)

Vector([13.0, 24.0, 35.0, 6.0])

Đọc chương này ko hay